In [6]:
import pandas as pd

df = pd.read_csv(".\\results.csv")
df

,Test name,Execution time,Result,Values
0,/20220315-MathProblems/STC_0001.smt2,0.003764,SAT,x = -100 y = 1 z = 100
1,/20220315-MathProblems/STC_0002.smt2,0.323468,SAT,x = -47 y = -24 z = 49
2,/20220315-MathProblems/STC_0003.smt2,0.573846,SAT,x = -5 y = 4 z = 4
3,/20220315-MathProblems/STC_0004.smt2,1.253370,Not SAT,NaN
4,/20220315-MathProblems/STC_0005.smt2,1.565967,Not SAT,NaN
...,...,...,...,...
995,/20220315-MathProblems/STC_0996.smt2,0.549865,SAT,x = -7 y = 2 z = 11
996,/20220315-MathProblems/STC_0997.smt2,0.556982,SAT,x = -3 y = 8 z = 8
997,/20220315-MathProblems/STC_0998.smt2,0.533815,SAT,x = -9 y = -1 z = 12
998,/20220315-MathProblems/STC_0999.smt2,0.530791,SAT,x = -9 y = 0 z = 12
